In [7]:
!pip install requests bs4 pandas
!pip install pytesseract pillow opencv-python pandas
!pip install easyocr



  Obtaining dependency information for easyocr from https://files.pythonhosted.org/packages/bb/84/4a2cab0e6adde6a85e7ba543862e5fc0250c51f3ac721a078a55cdcff250/easyocr-1.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/f2/35/0858e9e71b36948eafbc5e835874b63e515179dc3b742cbe3d76bc683439/opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for python-bidi from https://files.pythonhosted.org/packages/95/41/d4dbc72b96e2eea3aeb9292707459372c8682ef039cd19fcac7e09d513ef/python_bidi-0.6.6-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for Shapely from https://files.pythonhosted.org/packages/93/5b/842022c00fbb051083c1c85430f3bb55565b7fd2d775f4f398c0ba8052ce/shapely-2.1.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyclipper from https://files.pythonhosted.org/packages/17/4b/a4cda18e8556d913ff75052585eb0d6585005

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


In [14]:
!pip install numpy==1.25.2


  Obtaining dependency information for numpy==1.25.2 from https://files.pythonhosted.org/packages/72/b2/02770e60c4e2f7e158d923ab0dea4e9f146a2dbf267fec6d8dc61d475689/numpy-1.25.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.5 MB 1.7 MB/s eta 0:00:10
   -- ------------------------------------- 1.0/15.5 MB 7.9 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/15.5 MB 20.0 MB/s eta 0:00:01
   --------------- ------------------------ 6.0/15.5 MB 29.7 MB/s eta 0:00:01
   ---------------------- ----------------- 8.9/15.5 MB 35.5 MB/s eta 0:00:01
   ----------------------- ---------------- 9.2/15.5 MB 31.1 MB/s eta 0:00:01
   --------------------------------- ------ 13.0/15.5 MB 54.7 MB/s eta 0:00:01
   ------------------------------------- -- 14.7/15.5 MB 50.4 MB/s eta 0:00:01
   --------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
matplotlib 3.7.2 requires pyparsing<3.1,>=2.3.1, but you have pyparsing 3.2.3 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.25.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.2 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.2 which is incompatible.


In [3]:
#for afghanistan

import requests
import easyocr
from io import BytesIO
from PIL import Image
import pandas as pd
import re
import os

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Set the base URL and page range
base_url = "https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/"
page_range = range(4, 19)  

all_results = []

print("🔍 Starting OCR extraction...")
for i in page_range:
    url = f"{base_url}{i}.jpg"
    print(f"🔍 Processing page {i}: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))

        # OCR text
        results = reader.readtext(img, detail=0)
        all_results.extend([line.strip() for line in results if line.strip() != ""])
    except Exception as e:
        print(f"❌ Failed to process page {i}: {e}")


chunks = []
current_chunk = []

for line in all_results:
    if re.match(r"^\d{3,4}[A-Z]?", line):  
        if current_chunk:
            chunks.append(current_chunk)
        current_chunk = [line]
    else:
        current_chunk.append(line)

if current_chunk:
    chunks.append(current_chunk)


os.makedirs("debug", exist_ok=True)
with open("debug/chunks_debug.txt", "w") as f:
    for i, chunk in enumerate(chunks):
        f.write(f"=== Chunk {i} ===\n")
        f.write("\n".join(chunk))
        f.write("\n\n")

# Step 2: Define a parser
def parse_chunk(chunk):
    text = " ".join(chunk).replace(",", ".")  # normalize decimals
    scott_match = re.search(r"\b(\d{3,4}[A-Z]?)\b", text)
    year_match = re.search(r"\b(19\d{2})\b", text)
    prices = re.findall(r"\d+\.\d{2}", text)
    desc_color_match = re.search(
        r"\b\d{3,4}[A-Z]?\b\s+(.*?)([A-Za-z]{3,}.*?)?(?=\d{4}|\$|\d+\.\d{2}|$)",
        text
    )

    return {
        "Scott Number": scott_match.group(1) if scott_match else None,
        "Description": desc_color_match.group(1).strip() if desc_color_match else "",
        "Color": desc_color_match.group(2).strip() if desc_color_match and desc_color_match.lastindex >= 2 else "",
        "Year": year_match.group(1) if year_match else None,
        "Mint Price": prices[0] if len(prices) > 0 else None,
        "Used Price": prices[1] if len(prices) > 1 else None
    }


parsed_data = []
for chunk in chunks:
    parsed = parse_chunk(chunk)
    if parsed["Scott Number"]:
        parsed_data.append(parsed)

df = pd.DataFrame(parsed_data)
df.to_csv("afghanistan_stamps.csv", index=False)
print("✅ Done! Saved parsed data to 'afghanistan_stamps.csv'")




🔍 Starting OCR extraction...
🔍 Processing page 5: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/5.jpg
🔍 Processing page 6: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/6.jpg
🔍 Processing page 7: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/7.jpg
🔍 Processing page 8: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/8.jpg
🔍 Processing page 9: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/9.jpg
🔍 Processing page 10: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/10.jpg
🔍 Processing page 11: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/11.jpg
🔍 Processing page 12: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/12.jpg
🔍 Processing page 13: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/13.jpg
🔍 Processing page 14: https://reader.scottonline.com/sample/afghanistan/files/pages/tablet/14.jpg
🔍

In [5]:
import requests
import easyocr
from io import BytesIO
from PIL import Image
import pandas as pd
import re
import os
import time

# Initialize EasyOCR
reader = easyocr.Reader(['en'])

# Country list (no spaces in URLs)
countries = [
    "argentina", "austria", "belgium", "bulgaria", "canada",
    "chile", "colombia", "cuba", "france", "germany", "greatbritain",
    "greece", "hungary", "india", "jamaica", "japan", "korea", "malaysia",
    "mexico", "netherlands", "newzealand", "pakistan", "phillippines",
    "poland", "saudiarabia", "southafrica", "spain", "thailand", "turkey"
]

# Create folders
os.makedirs("parsed_stamps", exist_ok=True)
os.makedirs("debug_chunks", exist_ok=True)

def parse_chunk(chunk):
    text = " ".join(chunk).replace(",", ".")
    scott_match = re.search(r"\b(\d{3,4}[A-Z]?)\b", text)
    year_match = re.search(r"\b(19\d{2})\b", text)
    prices = re.findall(r"\d+\.\d{2}", text)
    desc_color_match = re.search(
        r"\b\d{3,4}[A-Z]?\b\s+(.*?)([A-Za-z]{3,}.*?)?(?=\d{4}|\$|\d+\.\d{2}|$)",
        text
    )
    return {
        "Scott Number": scott_match.group(1) if scott_match else None,
        "Description": desc_color_match.group(1).strip() if desc_color_match else "",
        "Color": desc_color_match.group(2).strip() if desc_color_match and desc_color_match.lastindex >= 2 else "",
        "Year": year_match.group(1) if year_match else None,
        "Mint Price": prices[0] if len(prices) > 0 else None,
        "Used Price": prices[1] if len(prices) > 1 else None
    }

def extract_country(country, page_range=range(1, 11)):
    base_url = f"https://reader.scottonline.com/sample/{country}/files/pages/tablet/"
    all_results = []

    print(f"🌍 Starting {country}")
    for i in page_range:
        url = f"{base_url}{i}.jpg"
        print(f"  📄 Page {i}: {url}")
        try:
            response = requests.get(url)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            results = reader.readtext(img, detail=0)
            all_results.extend([line.strip() for line in results if line.strip()])
        except Exception as e:
            print(f"  ❌ Failed to process page {i}: {e}")

    # Chunking lines by stamp entry
    chunks = []
    current_chunk = []

    for line in all_results:
        if re.match(r"^\d{3,4}[A-Z]?", line):
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = [line]
        else:
            current_chunk.append(line)
    if current_chunk:
        chunks.append(current_chunk)

    # Debug save
    with open(f"debug_chunks/{country}_chunks.txt", "w") as f:
        for i, chunk in enumerate(chunks):
            f.write(f"=== Chunk {i} ===\n")
            f.write("\n".join(chunk))
            f.write("\n\n")

    # Parse each chunk
    parsed_data = []
    for chunk in chunks:
        parsed = parse_chunk(chunk)
        if parsed["Scott Number"]:
            parsed_data.append(parsed)

    # Save DataFrame
    df = pd.DataFrame(parsed_data)
    df.to_csv(f"parsed_stamps/{country}_stamps.csv", index=False)
    print(f"✅ Done with {country}. Parsed {len(df)} entries.")

# Run for all countries
for country in countries:
    extract_country(country)
    time.sleep(1)  # Be kind to the server


🌍 Starting argentina
  📄 Page 1: https://reader.scottonline.com/sample/argentina/files/pages/tablet/1.jpg
  📄 Page 2: https://reader.scottonline.com/sample/argentina/files/pages/tablet/2.jpg
  📄 Page 3: https://reader.scottonline.com/sample/argentina/files/pages/tablet/3.jpg
  📄 Page 4: https://reader.scottonline.com/sample/argentina/files/pages/tablet/4.jpg
  📄 Page 5: https://reader.scottonline.com/sample/argentina/files/pages/tablet/5.jpg
  📄 Page 6: https://reader.scottonline.com/sample/argentina/files/pages/tablet/6.jpg
  📄 Page 7: https://reader.scottonline.com/sample/argentina/files/pages/tablet/7.jpg
  📄 Page 8: https://reader.scottonline.com/sample/argentina/files/pages/tablet/8.jpg
  📄 Page 9: https://reader.scottonline.com/sample/argentina/files/pages/tablet/9.jpg
  📄 Page 10: https://reader.scottonline.com/sample/argentina/files/pages/tablet/10.jpg
✅ Done with argentina. Parsed 230 entries.
🌍 Starting austria
  📄 Page 1: https://reader.scottonline.com/sample/austria/files/p

KeyboardInterrupt: 